1) Instalação de dependências

In [1]:
# (1) Atualiza apt e instala tesseract + poppler-utils
!sudo apt-get update
!sudo apt-get install -y poppler-utils tesseract-ocr

# (2) Instala todos os pacotes Python de uma vez
%pip install --quiet --upgrade \
    langchain-text-splitters langchain-community langgraph \
    langchain-openai langchain-core pypdf unstructured \
    youtube-transcript-api pdfminer.six \
    unstructured_inference pdf2image \
    pytesseract pillow unstructured-pytesseract \
    PyPDF2 pi-heif python-docx



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

2) Importações das bibliotecas necessarias

In [2]:
import os
import time
from google.colab import userdata
from typing import Literal
import requests
import re

from langchain.chat_models import init_chat_model
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain import PromptTemplate, LLMChain

from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

3) Conficuraçãa da API e definição dos perfis de usuarios

In [3]:
# --------------------------------------------------------
# 3.1) Inicializa o LLM e memória
# --------------------------------------------------------
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_2')
client = init_chat_model("gpt-4o-mini", model_provider="openai")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="question"
)

# --------------------------------------------------------
# 3.2) Definição de perfis de usuário
# --------------------------------------------------------
PERFIS_INFO = {
    'vitima': {
        'tipo_info': 'Resgate, primeiros socorros, rotas seguras.',
        'tom': 'Curto, empático e urgente.',
        'prioridade': 'Muito alta. Respostas devem focar na segurança imediata e clareza.',
    },
    'morador': {
        'tipo_info': 'Alertas meteorológicos, preparação e evacuação.',
        'tom': 'Explicativo e preventivo.',
        'prioridade': 'Alta. Deve ajudar a evitar riscos e oferecer orientações claras.',
    },
    'familiar': {
        'tipo_info': 'Localização de vítimas, contatos úteis, abrigos e hospitais.',
        'tom': 'Informativo, empático e tranquilizador.',
        'prioridade': 'Alta. Reforce caminhos seguros para encontrar a pessoa e acalmar o usuário.',
    }
}

<ipython-input-3-9bf5061b55e4>:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


3) Carrega e indexa documentos

In [4]:
PASTA_DOCUMENTOS = "/content/docsw"

docs = []
if os.path.exists(PASTA_DOCUMENTOS):
    for nome_arquivo in os.listdir(PASTA_DOCUMENTOS):
        caminho_completo = os.path.join(PASTA_DOCUMENTOS, nome_arquivo)

        if nome_arquivo.lower().endswith(".txt"):
            loader = TextLoader(caminho_completo, encoding="utf-8")
            docs.extend(loader.load())

        elif nome_arquivo.lower().endswith(".docx"):
            loader = UnstructuredWordDocumentLoader(caminho_completo)
            try:
                docs.extend(loader.load())
            except Exception as e:
                print(f"Erro ao carregar o DOCX {nome_arquivo}: {e}")

        else:
            print(f"Ignorando {nome_arquivo}: formato não suportado por este exemplo.")
else:
    print(f"A pasta de documentos {PASTA_DOCUMENTOS} não foi encontrada.")

# divisão em chunks e criação do vectorstore:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
documentos_chunked = []
for documento in docs:
    if isinstance(documento.page_content, str):
        textos_divididos = text_splitter.split_text(documento.page_content)
        for chunk in textos_divididos:
            documentos_chunked.append(
                documento.__class__(page_content=chunk, metadata=documento.metadata)
            )
    else:
        print(f"Ignorando documento com conteúdo não string: {documento.metadata}")

print(f"Documents originais: {len(docs)}")
print(f"Total de chunks após divisão: {len(documentos_chunked)}")

if documentos_chunked:
    embeddings = OpenAIEmbeddings()
    vectorstore_real = InMemoryVectorStore.from_documents(
        documents=documentos_chunked,
        embedding=embeddings
    )

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=client,
        retriever=vectorstore_real.as_retriever(),
        memory=memory,
        verbose=False
    )
else:
    print("Não há documentos processados para criar o vector store e iniciar o chat.")


Documents originais: 8
Total de chunks após divisão: 1188


4) Funçoes auxiliares

In [5]:
# --------------------------------------------------------
# 4.1) Função para obter previsão de 3 dias (via wttr.in)
# --------------------------------------------------------
def obter_previsao_3_dias(local_padrao: str = "Sao Paulo") -> str:
    """
    Retorna uma string com a previsão para hoje, amanhã e depois de amanhã
    em formato: data / descrição ao meio-dia / mínima / máxima.
    Usa wttr.in em JSON (sem API key).
    """
    cidade = local_padrao.replace(" ", "%20")
    url = f"https://wttr.in/{cidade}?lang=pt&format=j1"
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        dados = resp.json()
    except Exception:
        return (
            "Desculpe, não consegui obter a previsão do tempo no momento. "
            "Tente novamente mais tarde."
        )

    # Extrai a lista de 3 dias: índices 0 (hoje), 1 (amanhã), 2 (depois de amanhã)
    dias = dados.get("weather", [])
    if len(dias) < 3:
        return "Não consegui encontrar dados completos de previsão para os próximos 3 dias."

    linhas = [f"🌦️ Previsão para os próximos 3 dias em {local_padrao}:\n"]
    for idx, dia in enumerate(dias[:3]):
        data = dia.get("date", "–")
        min_c = dia.get("mintempC", "–")
        max_c = dia.get("maxtempC", "–")
        # Pega a descrição ao meio-dia (posição aproximada)
        hora_meio = dia.get("hourly", [])
        desc_meio = "–"
        if len(hora_meio) > 4:
            desc_meio = hora_meio[4].get("weatherDesc", [{"value": "–"}])[0].get("value", "–")

        if idx == 0:
            rotulo = "HOJE"
        elif idx == 1:
            rotulo = "AMANHÃ"
        else:
            rotulo = "DEPOIS DE AMANHÃ"

        linhas.append(
            f"- {rotulo} ({data}): {desc_meio}, mínima {min_c}°C / máxima {max_c}°C"
        )

    return "\n".join(linhas)


4) Função para iniciar o chat com perfis e memória

In [18]:
# --------------------------------------------------------
# 4.1) Função principal de chat
# --------------------------------------------------------
def iniciar_chat_com_memoria():
    # 1) Pré-carrega a previsão de 3 dias e armazena numa variável (sem printar)
    previsao_3_dias = obter_previsao_3_dias(local_padrao="Sao Paulo")
    # (observação: não exibimos nesse momento, apenas guardamos para injetar no contexto)

    # 2) Em seguida, pede nome e perfil (fluxo original)
    print("👋 Olá! Sou um assistente virtual especializado em desastres naturais.\n")
    print("Digite seu **nome** seguido do número ou nome do seu perfil:")
    print(" 1 - Vítima: Você está em perigo imediato e precisa de ajuda.")
    print(" 2 - Morador: Você mora em região afetada ou sob risco.")
    print(" 3 - Familiar: Está procurando informações sobre alguém afetado.\n")

    opcoes_validas = {
        '1': 'vitima',
        'vitima': 'vitima',
        '2': 'morador',
        'morador': 'morador',
        '3': 'familiar',
        'familiar': 'familiar'
    }

    nome_usuario = ''
    perfil = ''

    while perfil == '':
        entrada = input("Digite seu nome e número do perfil (ex: Ana 2): ").strip().lower()
        partes = entrada.split()
        if len(partes) < 2:
            print("⚠️ Entrada incompleta. Escreva seu nome e perfil. Ex: João 1")
            continue

        *nome_parts, perfil_raw = partes
        nome_usuario = ' '.join(p.capitalize() for p in nome_parts)
        perfil = opcoes_validas.get(perfil_raw.strip(), '')

        if not perfil:
            print("⚠️ Perfil inválido. Tente novamente com 1, 2, 3 ou nome correspondente.")

    info = PERFIS_INFO[perfil]
    print(f"\n✅ Perfil selecionado: {perfil.upper()} — como posso te ajudar hoje, {nome_usuario}?\n"
          f"⚠️ Obs. IMPORTANTE: esta orientação não substitui contato com os serviços oficiais (Defesa Civil, Corpo de Bombeiros, SAMU etc.)."
          f"(Digite 'sair' para encerrar)\n")

    # 3) Loop de interação
    while True:
        mensagem = input(f"🧑‍💬 {nome_usuario}: ").strip()
        if mensagem.lower() in ['sair', 'exit', 'quit']:
            print(f"👋 Sessão encerrada. Se cuide, {nome_usuario}!")
            break

        texto_minus = mensagem.lower()

        # 4) Se o usuário perguntar sobre o clima, exibe a previsão que já está em 'previsao_3_dias'
        if ("previsão" in texto_minus and "tempo" in texto_minus) or ("clima" in texto_minus):
            print(f"\n🤖 Assistente (Previsão do Tempo):\n{previsao_3_dias}\n")
            continue

        # 5) Senão, monta a pergunta incluindo NO CONTEXTO a previsão já pré-carregada
        #    Assim, o LLM “saberá” a previsão, mas só irá usá-la se for relevante para a resposta.
        question_com_contexto = (
            f"{nome_usuario} ({perfil.upper()}): {mensagem}\n\n"
            f"— Contexto adicional (previsão de 3 dias):\n"
            f"{previsao_3_dias}\n\n"
            f"— Contexto do perfil: {info['tipo_info']} — {info['tom']} — {info['prioridade']}"
        )

        resposta = qa_chain.invoke({
            "question": question_com_contexto,
            "chat_history": memory.chat_memory.messages
        })
        print("\n🤖 Assistente:", resposta['answer'], "\n")
        time.sleep(1)


5) Testes simulando diferentes perfis e situações: enchentes, incêndios, terremotos

In [20]:
# --------------------------------------------------------
# 5.1) testando perfil "VITIMA", caso de enchente
# -------------------------------------------------------
#script de perguntas:
# esta tudo alagado e estou no telhado de casa, o que devo fazer?
# meus documentos estão dentro da casa na enchente, devo tentar salva-los?
# qual o telefone do Corpo de Bombeiros ou Defesa Civil
iniciar_chat_com_memoria()

👋 Olá! Sou um assistente virtual especializado em desastres naturais.

Digite seu **nome** seguido do número ou nome do seu perfil:
 1 - Vítima: Você está em perigo imediato e precisa de ajuda.
 2 - Morador: Você mora em região afetada ou sob risco.
 3 - Familiar: Está procurando informações sobre alguém afetado.

Digite seu nome e número do perfil (ex: Ana 2): DANILO 1

✅ Perfil selecionado: VITIMA — como posso te ajudar hoje, Danilo?
⚠️ Obs. IMPORTANTE: esta orientação não substitui contato com os serviços oficiais (Defesa Civil, Corpo de Bombeiros, SAMU etc.).(Digite 'sair' para encerrar)

🧑‍💬 Danilo: esta tudo alagado e estou no telhado de casa, o que devo fazer?

🤖 Assistente: Se você estiver no telhado de casa e tudo estiver alagado, é importante tomar algumas precauções:

1. **Mantenha a calma**: Tente não entrar em pânico, pois isso pode dificultar a tomada de decisões.

2. **Chame por ajuda**: Tente entrar em contato com o Corpo de Bombeiros ou defesa civil para informar sua s

In [22]:
# --------------------------------------------------------
# 5.2) testando perfil Vitima, caso de incêndios
# -------------------------------------------------------
#script de perguntas:
# minha casa esta pegando fogo o que devo fazer?
# eu queimei o braço e agora ?
iniciar_chat_com_memoria()

👋 Olá! Sou um assistente virtual especializado em desastres naturais.

Digite seu **nome** seguido do número ou nome do seu perfil:
 1 - Vítima: Você está em perigo imediato e precisa de ajuda.
 2 - Morador: Você mora em região afetada ou sob risco.
 3 - Familiar: Está procurando informações sobre alguém afetado.

Digite seu nome e número do perfil (ex: Ana 2): DANILO 1

✅ Perfil selecionado: VITIMA — como posso te ajudar hoje, Danilo?
⚠️ Obs. IMPORTANTE: esta orientação não substitui contato com os serviços oficiais (Defesa Civil, Corpo de Bombeiros, SAMU etc.).(Digite 'sair' para encerrar)

🧑‍💬 Danilo: minha casa esta pegando fogo o que devo fazer?

🤖 Assistente: Se a sua casa está pegando fogo, siga estas etapas:

1. **Notifique rapidamente**: Grite "Kaji da!" para alertar familiares e vizinhos e ligue para 119 imediatamente, informando o local e a situação.

2. **Tente apagar rapidamente (se seguro)**: Se o fogo ainda não se espalhou para o teto e você se sentir seguro, tente apaga

In [23]:
# --------------------------------------------------------
# 5.2) testando perfil "Morador", caso de enchente
# -------------------------------------------------------
#script de perguntas:
# Moro em uma area de risco e esta chovendo muito forte, o que eu faço?
# o que é um plano de evacuação?
# qual é a previsão do tempo, vai parar de chover ?
iniciar_chat_com_memoria()

👋 Olá! Sou um assistente virtual especializado em desastres naturais.

Digite seu **nome** seguido do número ou nome do seu perfil:
 1 - Vítima: Você está em perigo imediato e precisa de ajuda.
 2 - Morador: Você mora em região afetada ou sob risco.
 3 - Familiar: Está procurando informações sobre alguém afetado.

Digite seu nome e número do perfil (ex: Ana 2): DANILO 2

✅ Perfil selecionado: MORADOR — como posso te ajudar hoje, Danilo?
⚠️ Obs. IMPORTANTE: esta orientação não substitui contato com os serviços oficiais (Defesa Civil, Corpo de Bombeiros, SAMU etc.).(Digite 'sair' para encerrar)

🧑‍💬 Danilo: Moro em uma area de risco e esta chovendo muito forte, o que eu faço?

🤖 Assistente: Se você mora em uma área de risco e está chovendo muito forte, siga estas orientações:

1. **Mantenha-se alerta**: Fique atento às condições climáticas e busque informações sobre a situação na sua área.
2. **Considere evacuação**: Se houver risco de deslizamento, pense em deixar o local. Se não for po

In [25]:
# --------------------------------------------------------
# 5.2) testando perfil "Morador", caso de incêndios
# -------------------------------------------------------
#script de perguntas:
# estou vendo minha casa pegar fogo, devo entrar e tentar salvar meus pertences?
# devo avisar os vizinhos ?
iniciar_chat_com_memoria()

👋 Olá! Sou um assistente virtual especializado em desastres naturais.

Digite seu **nome** seguido do número ou nome do seu perfil:
 1 - Vítima: Você está em perigo imediato e precisa de ajuda.
 2 - Morador: Você mora em região afetada ou sob risco.
 3 - Familiar: Está procurando informações sobre alguém afetado.

Digite seu nome e número do perfil (ex: Ana 2): DANILO 2

✅ Perfil selecionado: MORADOR — como posso te ajudar hoje, Danilo?
⚠️ Obs. IMPORTANTE: esta orientação não substitui contato com os serviços oficiais (Defesa Civil, Corpo de Bombeiros, SAMU etc.).(Digite 'sair' para encerrar)

🧑‍💬 Danilo: estou vendo minha casa pegar fogo, devo entrar e tentar salvar meus pertences?

🤖 Assistente: Não, você não deve entrar na sua casa que está pegando fogo. É importante evacuar o local imediatamente e ligar para o Corpo de Bombeiros. Tentar salvar seus pertences pode colocar sua vida em risco. Priorize a sua segurança e a de sua família. 

🧑‍💬 Danilo: devo avisar os vizinhos ?

🤖 Assis

In [26]:
# --------------------------------------------------------
# 5.2) testando perfil "Familiares", caso de enchente
# -------------------------------------------------------
#script de perguntas:
# a area que minha mão mora esta alagada e não estou conseguido contado por telefone o que devo fazer?
# devo entrar na area alagada para procura-la ?
iniciar_chat_com_memoria()

👋 Olá! Sou um assistente virtual especializado em desastres naturais.

Digite seu **nome** seguido do número ou nome do seu perfil:
 1 - Vítima: Você está em perigo imediato e precisa de ajuda.
 2 - Morador: Você mora em região afetada ou sob risco.
 3 - Familiar: Está procurando informações sobre alguém afetado.

Digite seu nome e número do perfil (ex: Ana 2): danilo 3

✅ Perfil selecionado: FAMILIAR — como posso te ajudar hoje, Danilo?
⚠️ Obs. IMPORTANTE: esta orientação não substitui contato com os serviços oficiais (Defesa Civil, Corpo de Bombeiros, SAMU etc.).(Digite 'sair' para encerrar)

🧑‍💬 Danilo: a area que minha mão mora esta alagada e não estou conseguido contado por telefone o que devo fazer?

🤖 Assistente: Se a área onde sua mãe mora está alagada e você não consegue contato por telefone, aqui estão algumas recomendações:

1. **Verifique informações**: Tente acompanhar as notícias locais através da internet ou de emissoras de rádio, para obter atualizações sobre a situação

In [28]:
# --------------------------------------------------------
# 5.2) testando perfil "o	Familiares", caso de incêndios
# -------------------------------------------------------
#script de perguntas:
# vi na tv que a casa da minha irmã pegou fogo e não estou conseguindo contato com ela o que eu faço ?
# o que é serviços de quadro de mensagens de desastres?
iniciar_chat_com_memoria()

👋 Olá! Sou um assistente virtual especializado em desastres naturais.

Digite seu **nome** seguido do número ou nome do seu perfil:
 1 - Vítima: Você está em perigo imediato e precisa de ajuda.
 2 - Morador: Você mora em região afetada ou sob risco.
 3 - Familiar: Está procurando informações sobre alguém afetado.

Digite seu nome e número do perfil (ex: Ana 2): DANILO 3

✅ Perfil selecionado: FAMILIAR — como posso te ajudar hoje, Danilo?
⚠️ Obs. IMPORTANTE: esta orientação não substitui contato com os serviços oficiais (Defesa Civil, Corpo de Bombeiros, SAMU etc.).(Digite 'sair' para encerrar)

🧑‍💬 Danilo: vi na tv que a casa da minha irmã pegou fogo e não estou conseguindo contato com ela o que eu faço ?

🤖 Assistente: Se você viu que a casa da sua irmã pegou fogo e não consegue contato com ela, faça o seguinte:

1. **Mantenha a calma**: É importante não entrar em pânico para poder pensar claramente.

2. **Ligue imediatamente para o Corpo de Bombeiros**: Disque 193 (ligação gratuita) 